# Plotting Tutorial

This notebook provides an example of reading and manipulating tables using the `astropy` package as well as accessing specific columns in a table for plotting purposes. Example plots are shown at the end of the notebook.

1. [Reading in Data & Displaying in Notebook](#reading-in-data-display-in-notebook)
1. [Joining Tables](#joining-tables)
1. [Keeping Specific Columns](#keeping-specific-columns)
1. [Plotting](#plotting)

# Reading in Data & Displaying in Notebook <a class="anchor" id="reading-in-data-display-in-notebook"></a>

In [ ]:
from astropy.table import Table
from astropy.table import join
import matplotlib.pyplot as plt

pisco_data = Table.read('pisco_data.txt', format='ascii')
sn_data = Table.read('sn1a_list.csv')

In [ ]:
pisco_data.show_in_notebook(display_length=10)

In [ ]:
sn_data.show_in_notebook(display_length=10)

# Joining Tables <a class="anchor" id="joining-tables"></a>

When joining tables, it is smart to make a copy of the originally saved table data. This is important because
if an error is made when manipulating the data, a backup copy still exists that was not overwritten and reading in the data again is not necessary.

In this example, when using SN names as keys to join tables, naming conventions between tables need to be changed to match. Be careful when changing SN naming conventions - modifications for one convention may not work for other conventions. In this case, it was sufficient to simply add 'SN' to the beginning of the names in the pisco data. 

When finally joining the tables, the two columns by which python joins the tables must be temporarily renamed to match. It is then possible to join the tables by a single column title - in this example, 'key'.

In [ ]:
# Make table copies
pisco_data_copy = pisco_data.copy()
sn_data_copy = sn_data.copy()

# Match naming conventions
for row in pisco_data_copy:
    if len(row['SNNAME']) < 8:
        row['SNNAME'] = 'SN' + row['SNNAME']

# Rename columns to match
pisco_data_copy.rename_column('SNNAME', 'key')
sn_data_copy.rename_column('sn', 'key')

combined_table = join(sn_data_copy, pisco_data_copy, keys='key')
combined_table.show_in_notebook(display_length=10)

# Keeping Specific Columns <a class="anchor" id="keeping-specific-columns"></a>

When a table has many columns and you only want certain columns to remain, it is easiest to use `.keep_columns` 
and specifically choose the titles of the columns you wish to keep.

In [ ]:
combined_table.keep_columns(['key', 'z', 'ST', 'COL', 'Bmag', 'SFR', 'OH', 'HaEW'])
combined_table.show_in_notebook(display_length=10)

It is important to note that `.keep_columns` will not produce the same effect as indexing the table by column names. `.keep_columns` will remove all unspecified columns in the original table, however, indexing on the column names themselves will leave the original table unchanged. By indexing, you are simply accessing the data in those columns.

# Plotting <a class="anchor" id="plotting"></a>

Before plotting, the names of the parameters have been assigned from specific columns in the joined table above. 
It is not necessary to do this before making the plots, however, this was just to demonstrate which column
corresponded to which parameter. 

In [ ]:
# Define variables
stretch = combined_table['ST']
color = combined_table['COL']
app_mag = combined_table['Bmag']
z = combined_table['z']
sfr = combined_table['SFR']
oh = combined_table['OH']
haew = combined_table['HaEW']

# Initiate figure
f = plt.figure()
plt.subplots_adjust(top = 7)

# Populate six subplots on the figure initiated above
ax1 = plt.subplot(6, 1, 1)
ax1.scatter(sfr, stretch)
ax1.set_xscale('log')
ax1.set_xlim(0.0001)
ax1.set_xlabel('SFR')
ax1.set_ylabel('Stretch')

ax2 = plt.subplot(6, 1, 2)
ax2.scatter(oh, stretch)
ax2.set_xlabel('OH')
ax2.set_ylabel('Stretch')

ax3 = plt.subplot(6, 1, 3)
ax3.scatter(haew, stretch)
ax3.set_xlabel('HaEW')
ax3.set_ylabel('Stretch')

ax4 = plt.subplot(6, 1, 4)
ax4.scatter(z, stretch)
ax4.set_xlabel('z')
ax4.set_ylabel('Stretch')

ax5 = plt.subplot(6, 1, 5)
ax5.scatter(app_mag, stretch)
ax5.set_xlabel('Bmag')
ax5.set_ylabel('Stretch')

ax6 = plt.subplot(6, 1, 6)
ax6.scatter(color, stretch)
ax6.set_xlabel('COL')
ax6.set_ylabel('Stretch')

plt.show()

For a more detailed subplot tutorial, see the `matploblib` demonstration [here](https://matplotlib.org/3.1.0/gallery/subplots_axes_and_figures/subplots_demo.html).